In [11]:
from pathlib import Path

import pandas as pd
from pyproj import CRS

import bedrock.gi as brgi
from bedrock.gi.ags.read import ags3_to_dfs
from bedrock.gi.models import Project

In [2]:
cwd = Path.cwd()
gi_dir = cwd / "gi-data" / "kaitak"
gi_files = [
    gi_dir / "31241" / "GE9908.7.ags",
    gi_dir / "44751" / "GE-2005-03-57 rev0.ags",
    gi_dir / "47615" / "GE-2007-13-4 rev0.ags",
]
gi_files

[WindowsPath('c:/Users/joost/ReposWindows/bedrock-gi/sandbox/gi-data/kaitak/31241/GE9908.7.ags'),
 WindowsPath('c:/Users/joost/ReposWindows/bedrock-gi/sandbox/gi-data/kaitak/44751/GE-2005-03-57 rev0.ags'),
 WindowsPath('c:/Users/joost/ReposWindows/bedrock-gi/sandbox/gi-data/kaitak/47615/GE-2007-13-4 rev0.ags')]

In [3]:
crs = CRS(2326)
crs

<Projected CRS: EPSG:2326>
Name: Hong Kong 1980 Grid System
Axis Info [cartesian]:
- N[north]: Northing (metre)
- E[east]: Easting (metre)
Area of Use:
- name: China - Hong Kong - onshore and offshore.
- bounds: (113.76, 22.13, 114.51, 22.58)
Coordinate Operation:
- name: Hong Kong 1980 Grid
- method: Transverse Mercator
Datum: Hong Kong 1980
- Ellipsoid: International 1924
- Prime Meridian: Greenwich

In [4]:
with open(gi_files[1]) as ags_file:
    ags_data = ags_file.read()

ags_data

'"**PROJ"\n"*PROJ_ID","*PROJ_NAME","*PROJ_LOC","*PROJ_CLNT","*PROJ_CONT","*PROJ_ENG","*PROJ_MEMO","*PROJ_DATE","*PROJ_AGS","*FILE_FSET"\n"<UNITS>","","","","","","","dd/mm/yyyy","",""\n"GE/2005/03.57","Ground Investigation - Urban (Term Contract)","Agreement No. KDO 01/2006 Site Investigation and Contamination Assessment at Remaining Area of Former Kai Tak Airport and Proposed Cruise Terminal","","","","","","",""\n"<CONT>","","","GEO - Geotechnical Projects Division (GI1)","Fugro Geotechnical Services Limited","Meinhardt Consulting Engineers","GE/2005/03","11/05/2007","3",""\n\n"**HOLE"\n"*HOLE_ID","*HOLE_TYPE","*HOLE_NATE","*HOLE_NATN","*HOLE_GL","*HOLE_FDEP","*HOLE_STAR","*HOLE_LOG","*?HOLE_DLOG","*?HOLE_CHEK","*?HOLE_DCHK","*HOLE_REM","*HOLE_ETRV","*HOLE_NTRV","*HOLE_LTRV","*HOLE_LETT","*HOLE_LOCX","*HOLE_LOCY",\n"*HOLE_LOCZ","*?HOLE_FLSH","*HOLE_ENDD","*HOLE_BACD","*HOLE_CREW","*HOLE_ORNT","*HOLE_INCL","*HOLE_EXC","*HOLE_SHOR","*HOLE_STAB","*HOLE_DIML","*HOLE_DIMW","*HOLE_LOCM","*

In [5]:
ags3_dfs = ags3_to_dfs(ags_data)

No data was found on line 5. Last Group: PROJ
No data was found on line 155. Last Group: HOLE
No data was found on line 199. Last Group: CDIA
No data was found on line 210. Last Group: CORE
No data was found on line 215. Last Group: DETL
No data was found on line 216. Last Group: DETL
No data was found on line 217. Last Group: DETL
No data was found on line 218. Last Group: DETL
No data was found on line 219. Last Group: DETL
No data was found on line 220. Last Group: DETL
No data was found on line 225. Last Group: FLSH
No data was found on line 226. Last Group: FLSH
No data was found on line 380. Last Group: GEOL
No data was found on line 424. Last Group: HDIA
No data was found on line 425. Last Group: HDIA
No data was found on line 426. Last Group: HDIA
No data was found on line 427. Last Group: HDIA
No data was found on line 428. Last Group: HDIA
No data was found on line 429. Last Group: HDIA
No data was found on line 430. Last Group: HDIA
No data was found on line 520. Last Group:

In [7]:
bedrock_dfs = {}

# Create the Bedrock Project table
if len(ags3_dfs["PROJ"]) != 1:
    raise ValueError("The PROJ group must contain exactly one row.")
else:
    ags_project = ags3_dfs["PROJ"].squeeze()
    br_project = Project(
        project_uid=ags_project["PROJ_ID"],
        crs=crs.to_wkt(),
    )
    project = pd.concat([pd.Series(br_project.model_dump()), ags_project])
    bedrock_dfs = {"Project": project}

bedrock_dfs["project"]

project_uid                                        GE/2005/03.57
crs            PROJCRS["Hong Kong 1980 Grid System",BASEGEOGC...
PROJ_ID                                            GE/2005/03.57
PROJ_NAME           Ground Investigation - Urban (Term Contract)
PROJ_LOC       Agreement No. KDO 01/2006 Site Investigation a...
PROJ_CLNT             GEO - Geotechnical Projects Division (GI1)
PROJ_CONT                    Fugro Geotechnical Services Limited
PROJ_ENG                          Meinhardt Consulting Engineers
PROJ_MEMO                                             GE/2005/03
PROJ_DATE                                             11/05/2007
PROJ_AGS                                                       3
FILE_FSET                                                       
dtype: object

In [ ]:
# Create the Bedrock Location table
if 

In [8]:
ags3_dfs["GEOL"]

,HOLE_ID,GEOL_TOP,GEOL_BASE,GEOL_DESC,GEOL_LEG,GEOL_GEOL,GEOL_GEO2,GEOL_STAT,FILE_FSET
0,B1,0.00,0.60,CONCRETE.,CONCRETE,Q,,,
1,B1,0.60,2.10,"Light grey (10R/7/1), fine to medium SAND with...",FILL,Q,,,
2,B1,2.10,4.10,"Stiff, yellowish brown (10YR/5/6), sandy, SILT...",FILL,Q,,,
3,B1,4.10,6.00,Wash boring.,WASHING,Q,,,
4,B10,0.00,0.40,CONCRETE.,CONCRETE,Q,,,
...,...,...,...,...,...,...,...,...,...
145,B8,3.40,3.90,"Stiff, grey (7.5YR/6/1) to dark grey (5R/4/1),...",FILL,Q,,,
146,B8,3.90,6.00,Wash boring.,WASHING,Q,,,
147,B9,0.00,0.80,CONCRETE.,CONCRETE,Q,,,
148,B9,0.80,4.30,"Dense, light grey (10R/7/1), silty, fine to co...",FILL,Q,,,


In [8]:
project_uid = bedrock_dfs["project"].project_uid

in_situ_groups = []
lab_groups = []
for group in ags3_dfs.keys():
    if group == "PROJ":
        continue
    else:
        ags3_dfs[group].insert(0, "project_uid", project_uid)

    if "HOLE_ID" in ags3_dfs[group].columns:
        in_situ_groups.append(group)

        ags3_dfs[group].rename(columns={"HOLE_ID": "source_id"}, inplace=True)

    if "SAMP_ID" in ags3_dfs[group].columns:
        lab_groups.append(group)
        ags3_dfs[group].rename(columns={"SAMP_ID": "source_id"}, inplace=True)

     project_uid HOLE_ID HOLE_TYPE  HOLE_NATE  HOLE_NATN HOLE_GL HOLE_FDEP  \
0  GE/2005/03.57      B1      IP+W  840359.23  818360.37    5.26      6.00   
1  GE/2005/03.57     B10      IP+W  839603.65  819200.43    4.61      6.00   

    HOLE_STAR  HOLE_LOG  ?HOLE_DLOG  ... HOLE_INCL HOLE_EXC HOLE_SHOR  \
0  07/02/2007  P. Zhang  08/02/2007  ...        90   FDR-03             
1  03/02/2007  P. Zhang  08/02/2007  ...        90   FDR-20             

  HOLE_STAB HOLE_DIML HOLE_DIMW HOLE_LOCM HOLE_LOCA HOLE_CLST FILE_FSET  
0                                                                        
1                                                                        

[2 rows x 35 columns]
     project_uid HOLE_ID CDIA_CDEP CDIA_HOLE CDIA_REM
0  GE/2005/03.57      B1      6.00       140         
1  GE/2005/03.57     B10      6.00       140         
     project_uid HOLE_ID CORE_TOP CORE_BOT CORE_PREC CORE_SREC CORE_RQD  \
0  GE/2005/03.57      B2     3.80     4.30        80           

# Speckle Stuff

In [ ]:
import datetime as dt

import pandas as pd
from dotenv import load_dotenv
from specklepy.api import operations
from specklepy.api.client import SpeckleClient
from specklepy.api.credentials import get_default_account
from specklepy.objects import Base
from specklepy.objects.geometry import Line, Point
from specklepy.transports.server import ServerTransport


def to_speckle():
    load_dotenv()
    stream_id = "7fbe8ed384"
    hole_table_path = "data/1_split2/HOLE.csv"
    df = pd.read_csv(hole_table_path, index_col=0)
    client, stream_id = get_stream(stream_id)

    # next create a server transport - this is the vehicle through which you will send and receive
    transport = ServerTransport(client=client, stream_id=stream_id)

    hash = create_hash(df, transport)

    commit_hash(hash, client, stream_id)


def get_stream(stream_id):
    # Authenticate with Speckle server
    speckle_server = "app.speckle.systems"
    speckle_token = os.environ["speckle_token"]
    client = SpeckleClient(host=speckle_server)
    account = get_default_account()

    client.authenticate_with_token(speckle_token)

    # create a new stream. this returns the stream id
    if not stream_id:
        stream_id = client.stream.create(name="a shiny new stream")

    # use that stream id to get the stream from the server
    new_stream = client.stream.get(id=stream_id)
    return client, stream_id


def create_hash(df, transport):
    newObj = Base()
    for i, row in df.iterrows():
        x = row["HOLE_NATE"]
        y = row["HOLE_NATN"]
        z_top = row["HOLE_GL"]
        z_bot = z_top - row["HOLE_FDEP"]

        # GisPointElement, GisLineElement lijken niet te werken
        p1 = Point(x=x, y=y, z=z_top)
        p2 = Point(x=x, y=y, z=z_bot)
        line = Line(start=p1, end=p2)
        relevant_cols = [
            "HOLE_ID",
            "HOLE_TYPE",
            "HOLE_STAR",
            "HOLE_LOG",
            "?HOLE_DLOG",
            "?HOLE_CHEK",
            "?HOLE_DCHK",
            "HOLE_REM",
            "?HOLE_FLSH",
            "HOLE_ENDD",
            "HOLE_BACD",
            "HOLE_CREW",
            "HOLE_INCL",
            "HOLE_EXC",
        ]

        for col in relevant_cols:
            line[col] = row[col]

        newObj[f"myline{i}"] = line
        break

    # this serialises the block and sends it to the transport
    hash = operations.send(base=newObj, transports=[transport])
    return hash


def commit_hash(hash, client, stream_id):
    # you can now create a commit on your stream with this object
    commid_id = client.commit.create(
        stream_id=stream_id,
        object_id=hash,
        message=f"these are lines I made in speckle-py at {dt.datetime.now()}",
    )


if __name__ == "__main__":
    to_speckle()
